## Required Installation

In [1]:
FT_MODE = "raft"

In [2]:
!pip install -q -U transformers peft accelerate optimum
!pip install datasets==2.15.0
!pip install trl
!pip install bitsandbytes
!pip install -q wandb
!pip install faiss-gpu
!pip install faiss-cpu
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 12.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninst

In [ ]:
from huggingface_hub import login

hf_token = ""
login(token=hf_token)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:



KeyboardInterrupt: 

# Load model and tokenizer from huggingface

In [ ]:
from peft import prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig

model_id = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load RAFT dataset

In [ ]:
from datasets import Dataset
dataset_path = "/content/drive/MyDrive/Colab Notebooks/RAFT/dataset/hotpotqa/train/data-00000-of-00001.arrow"
ds = Dataset.from_file(dataset_path)

In [ ]:
import pandas as pd
df = ds.to_pandas()

In [ ]:
def make_messages_for_training(row):
    if FT_MODE == "raft":
      user_content = row["instruction"]
      return [
          {"role": "user", "content": user_content},
          {"role": "assistant", "content": row["cot_answer"]},
      ]

    elif FT_MODE == "ft":
        user_content = (
            f"Question: {row["question"]}\n"
            "Answer the question briefly and directly in one short phrase."
        )
        return [
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": row["gold_answers"][0]},
        ]


df["messages"] = df.apply(make_messages_for_training, axis=1)


In [ ]:
df

,id,type,question,context,oracle_context,cot_answer,instruction,gold_answers,messages
0,hotpot_train_0,general,Which magazine was started first Arthur's Maga...,{'sentences': [['Radio City is India's first p...,Arthur's Magazine (1844–1846) was an American ...,assistant: To determine which magazine was sta...,<DOCUMENT>Radio City is India's first private ...,[Arthur's Magazine],"[{'role': 'user', 'content': '<DOCUMENT>Radio ..."
1,hotpot_train_1,general,The Oberoi family is part of a hotel company t...,{'sentences': [['Future Fibre Technologies (FF...,The Oberoi family is an Indian family that is ...,assistant: To answer the question about the lo...,<DOCUMENT>Future Fibre Technologies (FFT) is a...,[Delhi],"[{'role': 'user', 'content': '<DOCUMENT>Future..."
2,hotpot_train_2,general,Musician and satirist Allie Goertz wrote a son...,{'sentences': [['In addition to the show's reg...,"Allison Beth ""Allie"" Goertz (born March 2, 199...",assistant: To answer the question about whom M...,<DOCUMENT>In addition to the show's regular ca...,[President Richard Nixon],"[{'role': 'user', 'content': '<DOCUMENT>In add..."
3,hotpot_train_3,general,What nationality was James Henry Miller's wife?,{'sentences': [['James Henry Deakin (1851 – 8 ...,James Henry Miller (25 January 1915 – 22 Octob...,assistant: To determine the nationality of Jam...,<DOCUMENT>James Henry Deakin (1851 – 8 Novembe...,[American],"[{'role': 'user', 'content': '<DOCUMENT>James ..."
4,hotpot_train_4,general,Cadmium Chloride is slightly soluble in this c...,{'sentences': [['Cadmium chloride is a white c...,Cadmium chloride is a white crystalline compou...,"assistant: To answer the question, ""Cadmium Ch...",<DOCUMENT>Cadmium chloride is a white crystall...,[alcohol],"[{'role': 'user', 'content': '<DOCUMENT>Cadmiu..."
...,...,...,...,...,...,...,...,...,...
995,hotpot_train_1002,general,"Morrisson Kaye also spelled as Morrisson K, fo...","{'sentences': [['TPC Stonebrae, formerly TPC S...",Deep house is a subgenre of house music. It o...,assistant: To answer the question about the su...,"<DOCUMENT>TPC Stonebrae, formerly TPC San Fran...",[Deep house],"[{'role': 'user', 'content': '<DOCUMENT>TPC St..."
996,hotpot_train_1003,general,What type of organism can both Bellis and Syzy...,{'sentences': [['A predictive adaptive respons...,Syzygium jambos is a tree originating in South...,assistant: To determine what type of organism ...,<DOCUMENT>A predictive adaptive response (PAR)...,[plants],"[{'role': 'user', 'content': '<DOCUMENT>A pred..."
997,hotpot_train_1004,general,"Pandit Kishan Maharaj, was an Indian tabla pla...",{'sentences': [['Kumar Bose (born 4 April 1953...,Benares gharānā is one of the six most common ...,assistant: To answer the question about which ...,<DOCUMENT>Kumar Bose (born 4 April 1953) is an...,[Benares gharānā],"[{'role': 'user', 'content': '<DOCUMENT>Kumar ..."
998,hotpot_train_1005,general,Who composed the original score and songs of t...,{'sentences': [['The Boy Who Could Fly is a 20...,Finding Neverland is the original soundtrack a...,assistant: To determine who composed the origi...,<DOCUMENT>The Boy Who Could Fly is a 2015 Amer...,[Jan A. P. Kaczmarek],"[{'role': 'user', 'content': '<DOCUMENT>The Bo..."


In [ ]:
import datasets
ds = datasets.Dataset.from_pandas(df)

# Chat Template for prompt

In [ ]:
import re
import random
from multiprocessing import cpu_count

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

column_names = list(ds.features)
raw_datasets = ds.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)

Applying chat template (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
raw_datasets["text"]

Column(['<bos><start_of_turn>user\n\n\n<DOCUMENT>Radio City is India\'s first private FM radio station and was started on 3 July 2001.  It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).  It plays Hindi, English and regional songs.  It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.  Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.  The Radio station currently plays a mix of Hindi and Regional music.  Abraham Thomas is the CEO of the company.</DOCUMENT>\n<DOCUMENT>The 2014–15 Ukrainian Hockey Championship was the 23rd season of the Ukrainian Hockey Championship.  Only four teams participated in the league this season, because of the instability in Ukraine and that most

## Create train/test split

In [ ]:
raw_datasets = raw_datasets.train_test_split(test_size=0.05)
# create the splits
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]


# Peft config

In [ ]:
from peft import LoraConfig, get_peft_model
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["k_proj","o_proj","q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Quantization Config

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype="auto",
    use_cache=False,  # gradient checkpointing するなら False
    device_map="auto",
    quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

# SFT Training

In [ ]:

from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
# from transformers import TrainingArguments

# path where the Trainer will save its checkpoints and logs
parts = dataset_path.split("/")
dataset_idx = parts.index("dataset")
dataset_name = parts[dataset_idx + 1]

output_dir = f'/content/drive/MyDrive/Colab Notebooks/RAFT/train/{FT_MODE}_{dataset_name}'

# based on config
training_args = SFTConfig(
    fp16=True, # specify bf16=True instead when training on GPUs that support bf16
    do_eval=True,
    eval_strategy="steps",
    eval_steps=5,
    gradient_accumulation_steps=64,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=5.0e-5,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=2,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    save_strategy="no",
    save_total_limit=None,
    seed=42,
    # max_length=tokenizer.model_max_length,
    # max_length=512,
    max_length=2048,
    report_to="wandb",
    run_name=f"{FT_MODE}_{dataset_name}",
)


In [ ]:
trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        processing_class=tokenizer,
        peft_config=peft_config,
    )

Adding EOS to train dataset:   0%|          | 0/950 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/950 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/950 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Using auto half precision backend


## Start Training

In [ ]:
train_result = trainer.train()
# model

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1}.
The following columns in the Training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 950
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 64
  Total optimization steps = 30
  Number of trainable parameters = 1,490,944
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,2.677700,2.573047,1.272335,948075.000000,0.554126
2,2.540200,2.506999,1.322423,1896150.000000,0.557002


The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 50
  Batch size = 1
The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 50
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)




## Save the finetuned_raft_model

In [ ]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/RAFT/train/raft_hotpotqa
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-3-1b-it/snapshots/dcc83ea841ab6100d6b47a070329e1ba4cf78752/config.json
Model config Gemma3TextConfig {
  "_sliding_window_pattern": 6,
  "architectures": [
    "Gemma3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": null,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "dtype": "bfloat16",
  "eos_token_id": [
    1,
    106
  ],
  "final_logit_softcapping": null,
  "head_dim": 256,
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 1152,
  "initializer_range": 0.02,
  "intermediate_size": 6912,
  "layer_types": [
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_attention",
    "full_attention",
    "sliding_attention",
    "sliding_attention",
    "sliding_att

('/content/drive/MyDrive/Colab Notebooks/RAFT/train/raft_hotpotqa/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/RAFT/train/raft_hotpotqa/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/RAFT/train/raft_hotpotqa/chat_template.jinja',
 '/content/drive/MyDrive/Colab Notebooks/RAFT/train/raft_hotpotqa/tokenizer.model',
 '/content/drive/MyDrive/Colab Notebooks/RAFT/train/raft_hotpotqa/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/RAFT/train/raft_hotpotqa/tokenizer.json')

## Use Query only for inference

In [ ]:
import torch

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a RAFT model you give answer based on context. You also give reason for why this is the answer.",
    },
    {"role": "user", "content": "What did Paul Graham believe was the future of web development?"},
]

# prepare the messages for the model
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# inference
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


user
You are a RAFT model you give answer based on context. You also give reason for why this is the answer.

What did Paul Graham believe was the future of web development?
model
According to Paul Graham, the future of web development, as he envisioned it in his 1996 book *The Future of the Web*, was centered around **“distributed, networked, and modular” web applications.**

**Reasoning:**

* **Distributed Applications:** Graham believed that the web would move beyond individual web pages and become a network of interconnected applications. He envisioned applications communicating with each other seamlessly, rather than relying on a single, centralized server.  This was driven by the increasing availability of commodity hardware and the decreasing cost of networks.
* **Networked Applications:**  He emphasized the importance of networked applications – applications that could communicate with each other to create complex and interactive experiences. This was about building systems tha

## Use Query along with retreived context for inference.
* This is taken from the test set we created earlier.

In [ ]:
context_and_query = """
\n<DOCUMENT>And it wasn't, so I would.\n\nIn the summer of 2012 my mother had a stroke, and the cause turned out to be a blood clot caused by colon cancer. The stroke destroyed her balance, and she was put in a nursing home, but she really wanted to get out of it and back to her house, and my sister and I were determined to help her do it. I used to fly up to Oregon to visit her regularly, and I had a lot of time to think on those flights. </DOCUMENT>\n<DOCUMENT>I had no idea. </DOCUMENT>\n<DOCUMENT>Around this time, in the spring of 2000, I had an idea. It was clear from our experience with Viaweb that web apps were the future. Why not build a web app for making web apps? Why not let people edit code on our server through the browser, and then host the resulting applications for them? [9] You could run all sorts of services on the servers that these applications could use just by making an API call: making and receiving phone calls, manipulating images, taking credit card payments, etc.\n\nI got so excited about this idea that I couldn't think about anything else. </DOCUMENT>\n
What did Paul Graham believe was the future of web development?
"""

In [ ]:
import torch

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a RAFT model you give answer based on context. You also give reason for why this is the answer.",
    },
    {"role": "user", "content": context_and_query},
]

# prepare the messages for the model
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# inference
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


user
You are a RAFT model you give answer based on context. You also give reason for why this is the answer.

<DOCUMENT>And it wasn't, so I would.

In the summer of 2012 my mother had a stroke, and the cause turned out to be a blood clot caused by colon cancer. The stroke destroyed her balance, and she was put in a nursing home, but she really wanted to get out of it and back to her house, and my sister and I were determined to help her do it. I used to fly up to Oregon to visit her regularly, and I had a lot of time to think on those flights. </DOCUMENT>
<DOCUMENT>I had no idea. </DOCUMENT>
<DOCUMENT>Around this time, in the spring of 2000, I had an idea. It was clear from our experience with Viaweb that web apps were the future. Why not build a web app for making web apps? Why not let people edit code on our server through the browser, and then host the resulting applications for them? [9] You could run all sorts of services on the servers that these applications could use just by ma

# Reference:
1. https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Mistral/Supervised_fine_tuning_(SFT)_of_an_LLM_using_Hugging_Face_tooling.ipynb

### for RAG

In [5]:
import numpy as np
import faiss
from datasets import load_from_disk, Dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from typing import List, Optional, Tuple, Dict, Any
import tqdm
import json
import time
import os

# 想定カラム（RAFT系 dataset と同じ）
# - question: str
# - context: {"title": [[...]], "sentences": [[doc1, doc2, ...]]}
# - oracle_context: str
# - instruction: str (RAFT input string; 非RAGで FT_MODE=raft のときに使ってもOK)


# =========================
# 2) モデル読み込み
# =========================


# =========================
# 3) 非RAGのときの docs 取り出し
# =========================
def docs_from_row(row: Dict[str, Any]) -> List[str]:
    """
    非RAG推論時に dataset row から docs を取り出す。
    RAFT用 (gold + dummy) が context に入っている前提。
    """
    if "context" in row and row["context"] is not None:
        ctx = row["context"]
        # ctx["sentences"] が [[doc1, doc2, ...]] の形のはず
        if isinstance(ctx, dict) and "sentences" in ctx and len(ctx["sentences"]) > 0:
            docs = ctx["sentences"][0]
            if isinstance(docs, list) and len(docs) > 0:
                return docs

    return []


# =========================
# 4) RAG 用 retriever
#    - ds に "embedding" & faiss index があればそれを使う
#    - 無ければ retrieval_text(無ければ oracle_context) で index 作る
# =========================

RAG_INDEX = None
RAG_EMBEDDINGS = None  # 必要なら保持

# def ensure_retriever(
#     ds: Dataset,
#     embed_model_name="sentence-transformers/all-MiniLM-L6-v2",
#     text_column_candidates=("retrieval_text", "oracle_context"),
#     batch_size=64,
# ):
#     """
#     dsにembedding+faiss indexが無ければ作る。
#     戻り値: (ds_indexed, embed_model)
#     """
#     ds = ds.flatten_indices()

#     embed_model = SentenceTransformer(embed_model_name)

#     # 2) 既存の FAISS index があれば削除
#     try:
#         if ds.is_index_initialized("embedding"):
#             ds = ds.remove_faiss_index("embedding")
#     except Exception:
#         # まだ index がない場合は何もしない
#         pass

#     # 3) 古い embedding カラムがあれば削除
#     if "embedding" in ds.column_names:
#         ds = ds.remove_columns("embedding")

#     # 4) 使うテキストカラムを決める

#     texts = ds["retrieval_text"]

#     embeddings = embed_model.encode(
#         texts, batch_size=batch_size, show_progress_bar=True
#     )
#     embeddings = np.asarray(embeddings, dtype="float32")
#     faiss.normalize_L2(embeddings)

#     ds = ds.add_column("embedding", embeddings.tolist())

#     dim = embeddings.shape[1]
#     index = faiss.IndexFlatIP(dim)
#     index.add(embeddings)

#     ds = ds.add_faiss_index(column="embedding", custom_index=index)
#     return ds, embed_model


# グローバル変数にしておく（クラスにしてもOK）
RAG_INDEX = None
RAG_EMBEDDINGS = None  # 必要なら保持

def ensure_retriever(
    ds: Dataset,
    embed_model_name="BAAI/bge-base-en-v1.5",
    text_column_candidates=("retrieval_text", "oracle_context"),
    batch_size=64,
):
    """
    ds に対する埋め込みと FAISS インデックスを作って、
    ds（行数そのまま）と SentenceTransformer モデルを返す。
    """
    global RAG_INDEX, RAG_EMBEDDINGS

    # ① インデックスマッピングのズレを避けるため、最初に flatten
    ds = ds.flatten_indices()

    # ② モデル準備
    embed_model = SentenceTransformer(embed_model_name)

    # ③ テキストカラムを決める（とりあえず retrieval_text 前提）
    texts = ds["retrieval_text"]

    # ④ 埋め込み計算
    embeddings = embed_model.encode(
        texts, batch_size=batch_size, show_progress_bar=True
    )
    embeddings = np.asarray(embeddings, dtype="float32")
    faiss.normalize_L2(embeddings)

    # 安全チェック：行数とベクトル数が一致していることを確認
    assert embeddings.shape[0] == len(ds), (
        f"embeddings.shape[0]={embeddings.shape[0]} と len(ds)={len(ds)} が不一致です"
    )

    # ⑤ Dataset に埋め込みカラムを付ける（オプションだけどデバッグに便利）
    if "embedding" in ds.column_names:
        ds = ds.remove_columns("embedding")
    ds = ds.add_column("embedding", embeddings.tolist())

    # ⑥ FAISS index を自前で持つ
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)

    # 念のためチェック
    assert index.ntotal == len(ds), (
        f"index.ntotal={index.ntotal} と len(ds)={len(ds)} が不一致です"
    )

    RAG_INDEX = index
    RAG_EMBEDDINGS = embeddings  # 必要なら

    # ★ Dataset に add_faiss_index はしない！
    return ds, embed_model



# def retrieve_docs(ds_indexed: Dataset, embed_model: SentenceTransformer, query: str, top_k: int = 5):
#     """
#     ds_indexed 上の faiss index から top_k docs を取得
#     """
#     q_emb = embed_model.encode([query])
#     q_emb = np.asarray(q_emb, dtype="float32")
#     faiss.normalize_L2(q_emb)

#     scores, examples = ds_indexed.get_nearest_examples(
#         "embedding", q_emb[0], k=top_k
#     )
#     # examples は dict-of-lists
#     # retrieval_text or oracle_context を docs として返す
#     if "retrieval_text" in examples:
#         docs = examples["retrieval_text"]
#     else:
#         docs = examples["oracle_context"]
#     return docs, examples, scores


def retrieve_docs(ds_indexed: Dataset, embed_model: SentenceTransformer, query: str, top_k: int = 5):
    global RAG_INDEX
    assert RAG_INDEX is not None, "RAG_INDEX が初期化されていません（ensure_retriever を先に呼んでください）"

    # クエリ埋め込み
    q_emb = embed_model.encode([query])
    q_emb = np.asarray(q_emb, dtype="float32")
    faiss.normalize_L2(q_emb)

    # FAISS で検索
    scores, indices = RAG_INDEX.search(q_emb, top_k)  # shape: (1, top_k)
    idxs = indices[0].tolist()
    scs = scores[0].tolist()

    # Dataset から該当行を普通に取り出す
    docs = [ds_indexed[i]["retrieval_text"] for i in idxs]
    examples = ds_indexed.select(idxs)

    return docs, examples, scs



# =========================
# 5) 既存の build_inference_user_content を踏襲
# =========================
# def build_inference_user_content(
#     question: str,
#     docs: Optional[List[str]] = None,
#     oracle_context: Optional[str] = None
# ) -> str:
#     if FT_MODE == "raft":
#         context = ""
#         for d in docs or []:
#             context += f"<DOCUMENT>{d}</DOCUMENT>\n"
#         return context + question

#     elif FT_MODE == "ft":
#         return question

#     elif FT_MODE == "oracle":
#         return (oracle_context or "") + "\n\nQuestion: " + question

#     else:
#         raise ValueError(f"Unknown FT_MODE: {FT_MODE}")


def build_inference_user_content(
    question: str,
    docs: Optional[List[str]] = None,
    oracle_context: Optional[str] = None,
) -> str:

    context = ""
    for d in docs or []:
        context += f"<DOCUMENT>{d}</DOCUMENT>\n"

    if FT_MODE == "raft":
        prompt = f"""
            Question: {question}\nContext:\n {context}\n
            Answer this question using the information given in the context above. Here is things to pay attention to:
            - First provide step-by-step reasoning on how to answer the question.
            - In the reasoning, if you need to copy paste some sentences from the context, include them in ##begin_quote## and ##end_quote##. This would mean that things outside of ##begin_quote## and ##end_quote## are not directly copy paste from the context.
            - End your response with final answer in the form '<ANSWER>: <answer>', where <answer> is the actual answer and is brief and direct in one short phrase.
            You MUST begin your final answer with the tag "<ANSWER>:".
        """
        return prompt

    elif USE_RAG == False:
        # FTのみ / ベースラインでRAGコンテキストの付与をしないもの
        return (
            f"Question: {question}\n"
            "Answer the question briefly and directly in one short phrase."
        )

    else:
      # シンプルなFT / ベースラインにRAGのコンテキストを追加したもの
        return (
            f"Question: {question}\n"
            f"Context:\n{context}\n\n"
            "Answer the question briefly and directly in one short phrase by using the information given in the context above."
        )



# =========================
# 6) 推論1回分の関数（RAG / 非RAGの打ち分け含む）
# =========================
def infer_one(row: Dict[str, Any],
              ds_indexed: Optional[Dataset] = None,
              embed_model: Optional[SentenceTransformer] = None,
              top_k: int = 5,
              max_new_tokens: int = 1024):

    question = row["question"]
    device = torch.device("cuda")

    # --- docs / oracle_context をどう作るか ---
    if USE_RAG:
        # RAG: retrievalしたdocsを使う
        docs, ret_ex, scores = retrieve_docs(ds_indexed, embed_model, question, top_k=top_k)
        oracle_context = None  # RAG時は oracle を明示的に使わない想定
    else:
        # 非RAG: datasetのrowから inputを作る
        if FT_MODE == "raft":
            docs = docs_from_row(row)        # gold + dummy をrowから取得
            oracle_context = None
        # elif FT_MODE == "oracle":
        #     docs = None
        #     oracle_context = row.get("oracle_context", "")
        else:  # ft
            docs = None
            oracle_context = None

    user_content = build_inference_user_content(
        question,
        docs=docs,
        oracle_context=oracle_context
    )

    messages = [
        {
            "role": "system",
            "content": "The following is a conversation with an AI assistant. The assistant is helpful, clever, friendly and gives concise and accurate answers.",
        },
        {"role": "user", "content": user_content},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        truncation=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    device = torch.device("cuda")
    torch.cuda.reset_peak_memory_stats(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=False,
        )

    peak_bytes = torch.cuda.max_memory_allocated(device)
    peak_vram_mb = peak_bytes / (1024 ** 2)


    answer_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {
        "question": question,
        "answer_text": answer_text,
        "used_docs": docs if docs is not None else [],
        "use_rag": USE_RAG,
        "ft_mode": FT_MODE,
        "peak_vram_mb": peak_vram_mb,
    }


# =========================
# 7) 実行例
# =========================

USE_RAG = True          # True: RAG推論 / False: 非RAG推論
FT_MODE = "normal"        # [raft, ft, normal]
TOP_K = 5             # RAG の取得文書数

DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/RAFT/dataset/hotpotqa/validation/data-00000-of-00001.arrow"  # 既存データセットの保存先
DATASET_RAG_CORPUS_PATH = "/content/drive/MyDrive/Colab Notebooks/RAFT/dataset/hotpotqa/validation/hotpotqa_validation_rag_corpus"

parts = DATASET_PATH.split("/")
dataset_idx = parts.index("dataset")
dataset_name = parts[dataset_idx + 1]

OUTPUT_PATH = f"/content/drive/MyDrive/Colab Notebooks/RAFT/eval/{FT_MODE}_{dataset_name}_{USE_RAG}_predictions.jsonl"

if FT_MODE == "normal":
    MODEL_PATH = "google/gemma-3-1b-it"
else:
    MODEL_PATH = f'/content/drive/MyDrive/Colab Notebooks/RAFT/train/{FT_MODE}_{dataset_name}'




tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",      # dtype ではなく torch_dtype
    use_cache=False,         # gradient checkpointing するなら False のままでOK
    device_map="auto",
    quantization_config=bnb_config,
)

model.eval()

# ds: Dataset = load_from_disk(DATASET_PATH)
ds = Dataset.from_file(DATASET_PATH)
ds_rag_corpus = Dataset.load_from_disk(DATASET_RAG_CORPUS_PATH)

print("len(ds)           =", len(ds))
print("len(ds_rag_corpus) =", len(ds_rag_corpus))
if USE_RAG:
    ds_indexed, embed_model = ensure_retriever(ds_rag_corpus)
else:
    ds_indexed, embed_model = None, None

# 例: 先頭の1件で推論
# row0 = ds[0]
# out = infer_one(row0, ds_indexed=ds_indexed, embed_model=embed_model, top_k=TOP_K)

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    for i in tqdm.tqdm(range(len(ds))):
        row = ds[i]

        start = time.time()
        out = infer_one(row, ds_indexed=ds_indexed, embed_model=embed_model, top_k=TOP_K)
        end = time.time()

        record = {
            "id": row.get("id", i),
            "question": out["question"],
            "prediction": out["answer_text"],
            "gold_answer": row["gold_answers"],
            "used_docs": out["used_docs"],
            "use_rag": out["use_rag"],
            "ft_mode": out["ft_mode"],
            "time": end - start,
            "peak_vram_mb": out["peak_vram_mb"],
        }

        f.write(json.dumps(record, ensure_ascii=False) + "\n")


print("=== QUESTION ===")
print(out["question"])
print("\n=== ANSWER ===")
print(out["answer_text"])
print("\n=== USED DOCS (head) ===")
for d in out["used_docs"][:2]:
    print(d[:250], "...\n")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
100%|██████████| 300/300 [06:53<00:00,  1.38s/it]

=== QUESTION ===
What is the birthday of the actress who was the Duchess in "The Revengers Tragedy"?

=== ANSWER ===
user
The following is a conversation with an AI assistant. The assistant is helpful, clever, friendly and gives concise and accurate answers.

Question: What is the birthday of the actress who was the Duchess in "The Revengers Tragedy"?
Context:
<DOCUMENT>Revenge tragedy (less commonly referred to as revenge drama, revenge play, or tragedy of blood) defines a genre of plays made popular in early modern England.  Ashley H. Thorndike formally established this genre in his seminal 1902 article "The Relations of Hamlet to Contemporary Revenge Plays," which characterizes revenge tragedy "as a tragedy whose leading motive is revenge and whose main action deals with the progress of this revenge, leading to the death of the murderers and often the death of the avenger himself."  Thomas Kyd's "The Spanish Tragedy" (c.1580s) is often considered the inaugural revenge tragedy on the